In [56]:
import numpy as np
import cv2 as cv

img = cv.imread('data_hierarchy2.png')
img_white_bg = cv.imread('data_hierarchy3.png')

"""缩小图像,方便看效果
""" 
def resizeImg(src):
    height, width = src.shape[:2]
    size = (int(width * 0.3), int(height * 0.3))  # bgr
    img = cv.resize(src, size, interpolation=cv.INTER_AREA)
    return img

"""找出ROI，用于分割原图
原图有三块区域，一个是地块区域，一个是颜色示例区域，一个距离标尺区域
假设倒排后的轮廓列表是按照0地块，1标尺，2...N颜色排列
"""
def findROIContours(src):
    copy = src.copy()
    gray = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    # cv.imshow("gray", gray)
    
    # 低于thresh都变为黑色，maxval是给binary用的
    # 白底 254, 255 黑底 0, 255
    threshold = cv.threshold(gray, 0, 255, cv.THRESH_BINARY)[1]
    # cv.imshow("threshold", threshold)
    contours, hierarchy = cv.findContours(threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    sortedCnts = sorted(contours, key = cv.contourArea, reverse=True)
    # cv.drawContours(copy, [maxCnt], -1, (255, 0, 0), 2)
    # cv.imshow("roi contours", copy)
    return sortedCnts
    


"""按照mask，截取roi
"""
def getROIByContour(src, cnt):
    copy = src.copy()
    mask = np.zeros(copy.shape[:2], np.uint8)
    mask = cv.fillConvexPoly(mask, cnt, (255,255,255)) 
    # cv.imshow("mask", mask)
    # print(mask.shape)
    # print(copy.dtype)
    roi = cv.bitwise_and(copy, copy, mask=mask)
    # cv.imshow("roi", roi)
    return roi

"""找出所有的地块轮廓
"""
def findAllBlockContours(src):
    copy = src.copy()
    cv.imshow("findAllBlockContours copy", copy)
    gray = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    cv.imshow("findAllBlockContours gray", gray)
    # 这个canny的threshold到底该怎么计算？
    edges = cv.Canny(gray, 60, 150)
    cv.imshow("findAllBlockContours edges", edges)
    contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        if cv.contourArea(cnt) > 1000:
            cv.drawContours(copy, [cnt], -1, (255, 0, 0), 2)
            
    cv.imshow("findAllBlockContours", copy)
    return contours

img = resizeImg(img)
img_white_bg = resizeImg(img_white_bg)
sortedCnts = findROIContours(img)    
rootRegion = getROIByContour(img_white_bg, sortedCnts[0])
# blockCnts = findAllBlockContours(rootRegion)
print(len(sortedCnts[2:]))
for cnt in sortedCnts[2:]:
    epsilon = 0.01 * cv.arcLength(cnt, True)
    approxCurve = cv.approxPolyDP(cnt, epsilon, True)
    if len(approxCurve) == 4 and cv.contourArea(cnt) > 1000:
        colorRegion = getROIByContour(img_white_bg, cnt)
        # 这里截取出来的颜色块还是会有一些干扰在边上？怎么去掉？缩小矩形？
        cv.imshow("findColorRegions", colorRegion)
        print(len(approxCurve))
        print(cv.contourArea(cnt))
    
# # BGR转化为HSV
# HSV = cv.cvtColor(img, cv.COLOR_BGR2HSV)
# cv.imshow("imageHSV", HSV)
# cv.imshow('image', img)

# # 红色特殊[0,10], [156,180]
# color = [
#     # ([80, 0,0], [100, 255, 255])  # [b,g,r]
#     ([0, 43, 46], [10, 255, 255]),
#     ([156, 43, 46], [180, 255, 255])

# ]

# # 如果color中定义了几种颜色区间，都可以分割出来
# for (lower, upper) in color:
#     lower = np.array(lower, dtype="uint8")  # 颜色下限
#     upper = np.array(upper, dtype="uint8")  # 颜色上限
#     # 根据阈值找到对应颜色
#     mask = cv.inRange(HSV, lower, upper)  # 查找处于范围区间的
#     mask = 255 - mask  # 留下铝材区域
#     output = cv.bitwise_and(img, img, mask=mask)  # 获取铝材区域

#     #     output = cv.cvtColor(output,cv.COLOR_HSV2BGR)

#     # 展示图片
#     cv.imshow("images", np.hstack([img, output]))
#     contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     print(mask.shape)
#     # print(mask[0])
#     print(len(contours))
#     cv.drawContours(img, contours, -1, (0, 0, 255), 1)
#     for i in contours:
#         print(cv.contourArea(i))  # 计算缺陷区域面积
#         x, y, w, h = cv.boundingRect(i)  # 画矩形框
#         cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
#     # cv.imwrite(show_result_path, match_img_color)
#     cv.imshow("detect", img)
#     cv.imshow("chanle", img)
#     cv.waitKey(0)

cv.waitKey(0)
cv.destroyAllWindows()


38
4
1248.0
4
1248.0
4
1206.5
4
1196.0
4
1196.0
4
1196.0
